# Exploratory Data Analysis (EDA) #

### Task ###
- Perform a basic exploration of the cleaned data to understand its structure and key characteristics.

### Requirements ###

- Create a notebook (notebooks/EDA.ipynb) that includes:
- Basic summary statistics of the dataset (mean, variance, min, max, etc.).
- Visualization of feature distributions (histograms, box plots, etc.).
- Correlation analysis between different features and target variables (correlation matrix, heatmaps).
- Brief discussion on any patterns or insights observed during the analysis.
